# **📄 Document type classification baseline code**

**Contents**
- Prepare Environments
- Import Library & Define Functions
- Hyper-parameters
- Load Data
- Train Model
- Inference & Save File


# 1. Prepare Environments

* 데이터 로드를 위한 구글 드라이브를 마운트합니다.
* 필요한 라이브러리를 설치합니다.

In [ ]:
#%%time
# 필요한 라이브러리를 설치합니다. 1분걸림
#%pip install timm

In [ ]:
### 데이터 위치 셋팅
import os
os.chdir('/home')

Train_path = 'data/' +  'train_2_45D'
Train_csv = 'csv/'+     'train_2_45D.csv'

Test_path = 'data/' +   'test_final'
Test_csv = 'csv/' +     'sample_submission.csv'

Model_path = 'model/'+  'Swin2_train_2_45D'

## 2. Import Library & Define Functions

In [ ]:
import torch

# PyTorch 버전 확인
print(f"PyTorch version: {torch.__version__}")

# CUDA 사용 가능 여부 확인
print(f"Is CUDA available: {torch.cuda.is_available()}")

# CUDA 버전 확인
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")

# cuDNN 버전 확인
print(f"cuDNN version: {torch.backends.cudnn.version()}")

# GPU 이름 확인
if torch.cuda.is_available():
    print(f"GPU name: {torch.cuda.get_device_name(0)}")

PyTorch version: 2.1.0
Is CUDA available: True
CUDA version: 11.8
cuDNN version: 8700
GPU name: NVIDIA GeForce RTX 3090


In [ ]:

import time
import random

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
import cv2



In [ ]:
### 전역변수 설정
#home = '/content/drive/MyDrive/AILAB/CV/Contest/'

#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# 시드를 고정합니다.
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

In [ ]:
# 데이터셋 클래스를 정의합니다.
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)))
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

In [ ]:
# 학습을 위한 함수입니다.
def train_epoch(loader, model, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for image, targets in pbar:
        image = image.to(device)
        targets = targets.to(device)

        model.zero_grad(set_to_none=True)

        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
    }

    return ret

## 3. Hyper-parameters

In [ ]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model config
model_name = 'swinv2_base_window12_192' #swinv2_large_window12to24_192to384' # 'efficientnet_b5' #

# 하이퍼파라미터를 설정합니다.
img_size = 192  #224 #크기를 늘리면 모델이 더 많은 세부 정보를 학습할 수 있지만, 계산 비용이 증가합니다. #크기를 줄이면 계산 비용은 감소하지만, 정보 손실로 인해 성능이 떨어질 수 있습니다.
LR = 1e-4       #학습률이 높으면 학습이 빠르게 진행되지만, 최적값을 놓치거나 불안정할 수 있습니다. #학습률이 낮으면 안정적으로 학습되지만, 시간이 오래 걸릴 수 있습니다.
EPOCHS = 10     #에포크 수가 많으면 모델이 데이터에 대해 더 잘 학습할 수 있지만, 과적합(overfitting)의 위험이 있습니다. #에포크 수가 적으면 과소적합(underfitting)이 발생할 수 있습니다.
BATCH_SIZE = 4  #배치 크기가 크면 학습이 안정적이고, GPU의 효율성을 높일 수 있습니다. 하지만 메모리 사용량이 증가합니다. #배치 크기가 작으면 메모리 사용량이 적고, 미세한 업데이트가 가능하지만, 학습이 불안정할 수 있습니다.
num_workers = 2 #작업자 수가 많으면 데이터 로딩 속도가 빨라져 학습이 더 원활하게 진행될 수 있습니다. #너무 많은 작업자를 사용하면 시스템의 다른 작업과 충돌하거나 메모리 문제가 발생할 수 있습니다.

## 4. Load Data

In [ ]:
"""
# 데이터셋 경로 설정
dataset_path = Train_path
# 평균 및 표준편차 계산을 위한 변수 초기화
train_means = []
train_stds = []
# 이미지 경로에서 모든 이미지 파일을 읽어들임
image_files = [os.path.join(dataset_path, f) for f in os.listdir(dataset_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
for image_file in image_files:
    # 이미지 로드 및 RGB로 변환
    image = cv2.imread(image_file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # 이미지의 평균 및 표준편차 계산
    train_means.append(np.mean(image, axis=(0, 1)) / 255.0)
    train_stds.append(np.std(image, axis=(0, 1)) / 255.0)
# 전체 데이터셋의 평균 및 표준편차 계산
train_mean = np.mean(train_means, axis=0)
train_std = np.mean(train_stds, axis=0)
print(f"train_mean = {train_mean}")
print(f"train_std = {train_std}")

# 데이터셋 경로 설정
dataset_path = Test_path
# 평균 및 표준편차 계산을 위한 변수 초기화
test_means = []
test_stds = []
# 이미지 경로에서 모든 이미지 파일을 읽어들임
image_files = [os.path.join(dataset_path, f) for f in os.listdir(dataset_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
for image_file in image_files:
    # 이미지 로드 및 RGB로 변환
    image = cv2.imread(image_file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # 이미지의 평균 및 표준편차 계산
    test_means.append(np.mean(image, axis=(0, 1)) / 255.0)
    test_stds.append(np.std(image, axis=(0, 1)) / 255.0)
# 전체 데이터셋의 평균 및 표준편차 계산
test_mean = np.mean(test_means, axis=0)
test_std = np.mean(test_stds, axis=0)
print(f"test_mean = {test_mean}")
print(f"test_std = {test_std}")
"""

train_mean = [0.44537699, 0.44977048, 0.45193302]
train_std = [0.29171413, 0.29395023, 0.29496747]
test_mean = [0.44537699, 0.44977048, 0.45193302]
test_std = [0.29171413, 0.29395023, 0.29496747]
#test_mean = [0.49909188, 0.50301113, 0.50483625]
#test_std = [0.31717844, 0.31777685, 0.31768704]


# augmentation을 위한 transform 코드
trn_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=train_mean, std=train_std),
    ToTensorV2()
])

# test image 변환을 위한 transform 코드
tst_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=test_mean, std=test_std),
    ToTensorV2()
])

# 다시 확인하는데 걸리는 시간 :

In [ ]:
# 디렉토리 내 파일 개수 확인
def count_files_in_directory(directory):  # 디렉토리 내 파일 목록 가져오기
    files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    return len(files)  # 파일 개수 반환

# Dataset 정의
trn_dataset = ImageDataset(
    csv = Train_csv,
    path = Train_path,
    transform=trn_transform
)
tst_dataset = ImageDataset(
    Test_csv,
    Test_path,
    transform=tst_transform
)

print("train dataset 개수:", len(trn_dataset), "  test dataset 개수:", len(tst_dataset))

file_count1 = count_files_in_directory(trn_dataset.path)  # 수정된 변수 사용
file_count2 = count_files_in_directory(tst_dataset.path)  # 수정된 변수 사용
print(f"train 파일 개수: {file_count1}  test 파일 개수: {file_count2}")


# DataLoader 정의
trn_loader = DataLoader(
    trn_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=False
)
tst_loader = DataLoader(
    tst_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

train dataset 개수: 48000   test dataset 개수: 3140
train 파일 개수: 48000  test 파일 개수: 3140


## 5. Train Model

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
# load model
model = timm.create_model(
    model_name,
    pretrained=True,
    num_classes=17
).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LR)

print(f"model: {model_name}  |  img_size: {img_size}  |  Learning Rate: {LR}  |  EPOCHS: {EPOCHS}  |  BATCH_SIZE: {BATCH_SIZE}  |  num_workers: {num_workers}")

model: swinv2_base_window12_192  |  img_size: 192  |  Learning Rate: 0.0001  |  EPOCHS: 10  |  BATCH_SIZE: 4  |  num_workers: 2


In [ ]:
%%time
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
#브레이크
### 모델을 저장
# 메모리 가능 확인 기다리는 시간 30초

#from torch.utils.tensorboard import SummaryWriter

# TensorBoard SummaryWriter
#writer = SummaryWriter(log_dir='home/runs/exp1')

save_dir = Model_path
os.makedirs(save_dir, exist_ok=True)

for epoch in range(EPOCHS):
    ret = train_epoch(trn_loader, model, optimizer, loss_fn, device=device)  # train_epoch 함수를 호출하여 한 에포크 동안 모델을 학습시키고 결과를 반환받습니다.
    ret['epoch'] = epoch  # 반환된 결과 딕셔너리에 현재 에포크 번호를 추가합니다.

    log = ""  # 로그 메시지를 저장할 문자열 변수를 초기화합니다.
    for k, v in ret.items():  # 결과 딕셔너리의 각 항목을 순회하면서 키와 값을 로그 문자열에 추가합니다.
        log += f"{k}: {v:.4f}\n"  # 키와 값을 포맷팅하여 로그 문자열에 추가합니다.
    print(log)  # 완성된 로그 메시지를 출력합니다.

    # 모델의 상태를 저장합니다.
    checkpoint_path = os.path.join(save_dir, f"model_epoch_{epoch}.pth")
    torch.save(model.state_dict(), checkpoint_path)

    # TensorBoard에 기록합니다.
    #writer.add_scalar('Loss/train', ret['train_loss'], epoch)
    #writer.add_scalar('Accuracy/train', ret['train_acc'], epoch)
    #writer.add_scalar('F1_Score/train', ret['train_f1'], epoch)

#writer.close()"""

Loss: 0.5748:   0%|          | 59/12000 [00:08<27:14,  7.31it/s]


KeyboardInterrupt: 

### Model 2

In [ ]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model config
model_name = 'swinv2_large_window12to24_192to384' # 'efficientnet_b5' #

# 하이퍼파라미터를 설정합니다.
img_size = 384  #224 #크기를 늘리면 모델이 더 많은 세부 정보를 학습할 수 있지만, 계산 비용이 증가합니다. #크기를 줄이면 계산 비용은 감소하지만, 정보 손실로 인해 성능이 떨어질 수 있습니다.
LR = 1e-2       #학습률이 높으면 학습이 빠르게 진행되지만, 최적값을 놓치거나 불안정할 수 있습니다. #학습률이 낮으면 안정적으로 학습되지만, 시간이 오래 걸릴 수 있습니다.
EPOCHS = 5     #에포크 수가 많으면 모델이 데이터에 대해 더 잘 학습할 수 있지만, 과적합(overfitting)의 위험이 있습니다. #에포크 수가 적으면 과소적합(underfitting)이 발생할 수 있습니다.
BATCH_SIZE = 4  #배치 크기가 크면 학습이 안정적이고, GPU의 효율성을 높일 수 있습니다. 하지만 메모리 사용량이 증가합니다. #배치 크기가 작으면 메모리 사용량이 적고, 미세한 업데이트가 가능하지만, 학습이 불안정할 수 있습니다.
num_workers = 2 #작업자 수가 많으면 데이터 로딩 속도가 빨라져 학습이 더 원활하게 진행될 수 있습니다. #너무 많은 작업자를 사용하면 시스템의 다른 작업과 충돌하거나 메모리 문제가 발생할 수 있습니다.

In [ ]:
train_mean: [0.44537699, 0.44977048, 0.45193302]
train_std: [0.29171413, 0.29395023, 0.29496747]
test_mean = [0.49909188, 0.50301113, 0.50483625]
test_std: [0.31717844, 0.31777685, 0.31768704]


# augmentation을 위한 transform 코드
trn_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=train_mean, std=train_std),
    ToTensorV2()
])

# test image 변환을 위한 transform 코드
tst_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=test_mean, std=test_std),
    ToTensorV2()
])

In [ ]:
# 디렉토리 내 파일 개수 확인
def count_files_in_directory(directory):  # 디렉토리 내 파일 목록 가져오기
    files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    return len(files)  # 파일 개수 반환

# Dataset 정의
trn_dataset = ImageDataset(
    csv = Train_csv,
    path = Train_path,
    transform=trn_transform
)
tst_dataset = ImageDataset(
    Test_csv,
    Test_path,
    transform=tst_transform
)

print("train dataset 개수:", len(trn_dataset), "  test dataset 개수:", len(tst_dataset))

file_count1 = count_files_in_directory(trn_dataset.path)  # 수정된 변수 사용
file_count2 = count_files_in_directory(tst_dataset.path)  # 수정된 변수 사용
print(f"train 파일 개수: {file_count1}  test 파일 개수: {file_count2}")


# DataLoader 정의
trn_loader = DataLoader(
    trn_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=False
)
tst_loader = DataLoader(
    tst_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

train dataset 개수: 48000   test dataset 개수: 3140
train 파일 개수: 48000  test 파일 개수: 3140


In [ ]:
# load model
model = timm.create_model(
    model_name,
    pretrained=True,
    num_classes=17
).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LR)



In [ ]:
%%time
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
#브레이크
### 모델을 저장
# 메모리 가능 확인 기다리는 시간 30초

#from torch.utils.tensorboard import SummaryWriter

# TensorBoard SummaryWriter
#writer = SummaryWriter(log_dir='home/runs/exp1')

save_dir = Model_path + "swinv2_large_window12to24_192to384"
os.makedirs(save_dir, exist_ok=True)

for epoch in range(EPOCHS):
    ret = train_epoch(trn_loader, model, optimizer, loss_fn, device=device)  # train_epoch 함수를 호출하여 한 에포크 동안 모델을 학습시키고 결과를 반환받습니다.
    ret['epoch'] = epoch  # 반환된 결과 딕셔너리에 현재 에포크 번호를 추가합니다.

    log = ""  # 로그 메시지를 저장할 문자열 변수를 초기화합니다.
    for k, v in ret.items():  # 결과 딕셔너리의 각 항목을 순회하면서 키와 값을 로그 문자열에 추가합니다.
        log += f"{k}: {v:.4f}\n"  # 키와 값을 포맷팅하여 로그 문자열에 추가합니다.
    print(log)  # 완성된 로그 메시지를 출력합니다.

    # 모델의 상태를 저장합니다.
    checkpoint_path = os.path.join(save_dir, f"model_epoch_{epoch}.pth")
    torch.save(model.state_dict(), checkpoint_path)

    # TensorBoard에 기록합니다.
    #writer.add_scalar('Loss/train', ret['train_loss'], epoch)
    #writer.add_scalar('Accuracy/train', ret['train_acc'], epoch)
    #writer.add_scalar('F1_Score/train', ret['train_f1'], epoch)

#writer.close()"""

Loss: 2.5411: 100%|██████████| 12000/12000 [1:29:38<00:00,  2.23it/s]


train_loss: 2.9498
train_acc: 0.0681
train_f1: 0.0562
epoch: 0.0000



Loss: 3.0206: 100%|██████████| 12000/12000 [1:29:36<00:00,  2.23it/s]


train_loss: 2.9185
train_acc: 0.0664
train_f1: 0.0557
epoch: 1.0000



Loss: 3.2014:   2%|▏         | 211/12000 [01:35<1:28:30,  2.22it/s]


KeyboardInterrupt: 

In [ ]:
#torch.cuda.empty_cache()
#!nvidia-smi

# 6. Inference & Save File

In [ ]:
# 브레이크

### 저장된 모델 불러오기

# 에포크 4에서 저장된 모델 파일 경로
epoch_to_load = 5
checkpoint_path = os.path.join(Model_path, f"model_epoch_{epoch_to_load}.pth")

# 저장된 모델 상태를 불러옵니다
model.load_state_dict(torch.load(checkpoint_path))


<All keys matched successfully>

In [ ]:

preds_list = []

model.eval()
for image, _ in tqdm(tst_loader):
    image = image.to(device)

    with torch.no_grad():
        preds = model(image)
    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

100%|██████████| 785/785 [00:39<00:00, 19.86it/s]


In [ ]:
pred_df = pd.DataFrame(tst_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list

sample_submission_df = pd.read_csv("csv/sample_submission.csv")
assert (sample_submission_df['ID'] == pred_df['ID']).all()

pred_df.to_csv("csv/result_swin2_train_2_450"+str(epoch_to_load)+".csv", index=False)

In [ ]:
# pred_df Macro F1 score 계산
#from sklearn.metrics import f1_score
#f1_score(sample_submission_df['target'], pred_df['target'], average='macro')

In [ ]:
pred_df.head(5)

,ID,target
0,0008fdb22ddce0ce.jpg,2
1,00091bffdffd83de.jpg,12
2,00396fbc1f6cc21d.jpg,5
3,00471f8038d9c4b6.jpg,13
4,00901f504008d884.jpg,2


In [ ]:
pred_df['target'].value_counts()

In [ ]:
#ex = pred_df.reset_index()

ex1 = pred_df.loc[pred_df['target'] == 0]
ex2 = pred_df.loc[pred_df['target'] != 0]

In [ ]:
print(ex1, ex2)

                        ID  target
40    0412f6a5ba912add.jpg       0
57    05e33cb53c1ac02a.jpg       0
168   0f3bedf01f25f064.jpg       0
343   1dfce34ac940049d.jpg       0
439   267e2c2ec8182f3e.jpg       0
611   35131e3f1dc21609.jpg       0
616   3551ff70a2ee733b.jpg       0
821   43f11f0c5245ccb9.jpg       0
938   4f48d6bc2d3af344.jpg       0
1118  5eba163f304ef235.jpg       0
1123  5f0e93a989259358.jpg       0
1147  613ee74ac4a149c1.jpg       0
1226  670e484e9f4d4000.jpg       0
1269  6aa1e9befa8af3f8.jpg       0
1710  8f307c4c9f0ec2f8.jpg       0
1784  95c6c1c9dd14cad3.jpg       0
1904  9e32d5c65c99feac.jpg       0
1923  9f5b1c60ab6753cb.jpg       0
2007  a67caaccc2ea8ab2.jpg       0
2278  bbaec8719d4109d0.jpg       0
2375  c5c6626a8c92d45a.jpg       0
2405  c7fe39e0b08a84c4.jpg       0
2490  cf1937be84a9ba49.jpg       0
2548  d30cca5a80ea5a72.jpg       0
2556  d394c42a267a320b.jpg       0
2620  d86b9af8c6e19923.jpg       0
2705  de6ecb4369bfa015.jpg       0
2746  e1924ae57aaee6

In [ ]:
ex1.to_csv('/home/step1.csv')

In [ ]:
# 1부터 17까지의 숫자로 폴더 생성
for i in range(0, 17):
    folder_name = str(i)
    folder_path = os.path.join('/home/data/cl/', folder_name)
    os.makedirs(folder_path, exist_ok=True)
    #print(f"폴더 생성: {folder_path}")

class_df = pred_df

for i in range(0, len(class_df)) :
    print(class_df['ID'][i], class_df['target'][i] )
    img = cv2.imread('/home/data/test2x/' + class_df['ID'][i])
    cv2.imwrite('/home/data/cl/' + str(class_df['target'][i]) + '/' + class_df['ID'][i], img)



In [ ]:
### 데이터셋 결과 비교

filename = "result_swin2_train_2_450" + str(epoch_to_load)

# 데이터셋 로드
a_path = 'csv/' + str(filename) + '.csv'
b_path = 'csv/answer.csv'

a_df = pd.read_csv(a_path)
b_df = pd.read_csv(b_path)

# a_df.shape, b_df.shape:

b_df = b_df.drop(columns=['Unnamed: 0'])

print("비교할 파일 주소")
print(a_path)
print(b_path)

# 두 데이터프레임의 크기가 같은지 확인
if a_df.shape != b_df.shape:
    raise ValueError("두 데이터프레임의 크기가 다릅니다.")

# 두 데이터프레임의 각 셀을 비교
comparison = a_df != b_df

# 다른 항목의 개수를 계산
difference_count = comparison.sum().sum()

#print(comparison)

print(f"데이터 간 다른 항목수 : {difference_count}  / 3140")

print(f"데이터 중 맞은 퍼센트 : {(3140 - difference_count) / 3140 * 100}")


비교할 파일 주소
csv/result_swin2_train_2_4505.csv
csv/answer.csv
데이터 간 다른 항목수 : 558  / 3140
데이터 중 맞은 퍼센트 : 82.22929936305732


In [ ]:
비교할 파일 주소
csv/result_swin2_train_2_4509.csv
csv/answer.csv
데이터 간 다른 항목수 : 655  / 3140
데이터 중 맞은 퍼센트 : 79.14012738853503

비교할 파일 주소
csv/result_swin2_train_2_4505.csv
csv/answer.csv
데이터 간 다른 항목수 : 558  / 3140
데이터 중 맞은 퍼센트 : 82.22929936305732

In [ ]:
a=(0.9179*3140)-3140
print(a)

-257.79399999999987


In [ ]:
비교할 파일 주소
csv/result_1_padding.csv   #ep3임
csv/answer.csv
3140 rows x 2 columns]
데이터 간 다른 항목수 : 369  / 3140
데이터 중 맞은 퍼센트 : 88.2484076433121
epoch 3.0


비교할 파일 주소
csv/result_1_padding_ep2.csv
csv/answer.csv
[3140 rows x 2 columns]
데이터 간 다른 항목수 : 373  / 3140
데이터 중 맞은 퍼센트 : 88.12101910828025


비교할 파일 주소
csv/result_1_padding_ep4.csv
csv/answer.csv
[3140 rows x 2 columns]
데이터 간 다른 항목수 : 333  / 3140
데이터 중 맞은 퍼센트 : 89.39490445859872

In [ ]:
비교할 파일 주소
csv/result_2_bc_ep2.csv
csv/answer.csv
[3140 rows x 2 columns]
데이터 간 다른 항목수 : 374  / 3140
데이터 중 맞은 퍼센트 : 88.0891719745223

비교할 파일 주소
csv/result_2_bc_ep3.csv
csv/answer.csv
[3140 rows x 2 columns]
데이터 간 다른 항목수 : 333  / 3140
데이터 중 맞은 퍼센트 : 89.39490445859872

비교할 파일 주소
csv/result_2_bc_ep4.csv
csv/answer.csv
데이터 간 다른 항목수 : 320  / 3140
데이터 중 맞은 퍼센트 : 89.80891719745223

비교할 파일 주소
csv/result_2_bc_ep5.csv
csv/answer.csv
데이터 간 다른 항목수 : 386  / 3140
데이터 중 맞은 퍼센트 : 87.70700636942675

비교할 파일 주소
csv/result_2_bc_ep6.csv
csv/answer.csv
데이터 간 다른 항목수 : 330  / 3140
데이터 중 맞은 퍼센트 : 89.49044585987261

비교할 파일 주소
csv/result_2_bc_ep7.csv
csv/answer.csv
데이터 간 다른 항목수 : 314  / 3140
데이터 중 맞은 퍼센트 : 90.0 = 9179

비교할 파일 주소
csv/result_2_bc_ep8.csv
csv/answer.csv
데이터 간 다른 항목수 : 410  / 3140
데이터 중 맞은 퍼센트 : 86.94267515923568

비교할 파일 주소
csv/result_2_bc_ep9.csv
csv/answer.csv
데이터 간 다른 항목수 : 373  / 3140
데이터 중 맞은 퍼센트 : 88.12101910828025